# Lecture 11. ```joblib```

* Joblib provides a simple helper class to write **parallel `for` loops** using multiprocessing.

* The core idea is to write the code to be executed as a generator expression and convert it to parallel computing. 

In [3]:
import time
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import os

### Here, we ensure that we use one single CPU core in each operation. 
### By default, some numpy functions use multiple cores, e.g., the matrix multiplication. 
### If so, parallel computing with multiple CPUs is problematic. 

os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

In [4]:
def countdown(n):
    while n>0:
        n -= 1
    return n


t = time.time()
[countdown(10**7) for _ in range(20)]
print(time.time() - t)  

4.033647298812866


The above codes can be spread over **2 CPUs** using the following:

In [6]:
t = time.time()
results = Parallel(n_jobs=2)(delayed(countdown)(10**7) for _ in range(20))
#print(results)
print(time.time() - t)

2.1571218967437744


You may notice that the run time using 2 CPUs is more than one half of the run time using 1 CPU. 

This is due to the overhead when you use multiprocessing. 

#### Check the number of CPUs in your computer:

In [9]:
print(os.cpu_count())

8


#### Let's revisit two data analysis examples that we learnt in the previous lectures, and see how to use ```Parallel``` to speed up the codes.

## Example I: Perform the rank-transformation of the firm characteristics

In [12]:
D = pd.read_parquet('data/HK_stocks_151signals.parquet', engine='pyarrow')
D = D[(D.eom>='2000-01') & (D.eom<'2021-01')]
D = D.set_index(['id', 'eom'])

history_lengths = D.index.to_frame()['id'].groupby(level=0).count()
cs_idx_with_history = history_lengths[history_lengths >= 24].index 
D_full = D.loc[D.index.get_level_values(0).isin(cs_idx_with_history),:]

In [13]:
t = time.time()
D_ranked = D_full.groupby(level=1).rank(ascending=True)
D_ranked = D_ranked / D_ranked.groupby(level=1).max() - 0.5
print(time.time() - t)

9.990234851837158


#### How do we use ```joblib``` to speed up the above codes? 

In [15]:
def process_window(df):
    df_ranked = df.rank(ascending=True)
    df_ranked = df_ranked / df_ranked.max() - 0.5
    return df_ranked

In [16]:
t = time.time()

D_full2 = D_full.swaplevel()
ts_idx = D_full2.index.get_level_values(0).unique()

D_parts = Parallel(n_jobs=4)(delayed(process_window)(D_full2.loc[[ts_idx[t]]]) 
                              for t in range(0, len(ts_idx)))
D_ranked2 = pd.concat([i for i in D_parts], axis=0, keys=ts_idx)

#D_parts = Parallel(n_jobs=4)(delayed(process_window)(D_full2.loc[ts_idx[t]]) for t in range(len(ts_idx)))
#D_ranked = pd.concat(D_parts, axis=0, keys=ts_idx)

print(time.time() - t)

4.64065408706665


## Example II. Bootstrapping

In data analysis, we often want to quantify the estimation uncertainty in model parameters. For example, 
* what is the $90\%$ confidence intervals of the correlation between two stock returns?
* what is the $90\%$ confidence intervals of the market beta of AAPL?

Indeed, you can rely on some parametric models and compute the standard errors/confidence intervals based on large-sample asymptotic theory. 

Here, I would like to show you how to use the idea of Bootstrapping to properly quantify the estimation uncertainty in finite data samples. 

In [19]:
close_px_all = pd.read_csv('data/stock_px_2.csv',
                           parse_dates=True, index_col=0)
close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]
rets = close_px.loc['2006':'2010'][['AAPL', 'MSFT']].pct_change().dropna()
print(rets.shape)
rets.head()

(1258, 2)


,AAPL,MSFT
2006-01-04,0.002943,0.004568
2006-01-05,-0.007870,0.000827
2006-01-06,0.025813,-0.002891
2006-01-09,-0.003277,-0.002071
2006-01-10,0.063248,0.005396


In [20]:
rets.corr()    # how do we quantify the estimation uncertainty in the correlation coefficient?

,AAPL,MSFT
AAPL,1.000000,0.479162
MSFT,0.479162,1.000000


#### How to quantify the estimation uncertainty in the correlation between stock returns of AAPL and MSFT? -- Bootstrapping

Suppose that $T$ is the time-series sample size of the observed dataset. In our data, $T = 1258$ days. 

* First, we randomly draw (with replacement) a sample of $T$ days from ```rets```.
  
* Second, we compute the correlation coefficients between AAPL and MSFT for each randomly sampled dataset.
  
* Repeat the above two steps for 10,000 times, and report the mean and $90\%$ quantiles of the correlation coefficients.

In [23]:
sample_index = np.random.choice(a=np.arange(stop=rets.shape[0], dtype=np.int64), size=rets.shape[0])
print(sample_index)

[ 699  722  434 ...  562   63 1161]


In [54]:
print(rets.iloc[sample_index,:].shape)

(1258, 2)


In [24]:
rets.iloc[sample_index,:].corr().iloc[0,1]

0.4967313972006145

In [25]:
def get_corr_bootstrap(data_df, random_seed):
    np.random.seed(random_seed)
    sample_index = np.random.choice(a=np.arange(stop=data_df.shape[0], dtype=np.int64), size=data_df.shape[0])
    return data_df.iloc[sample_index,:].corr().iloc[0,1]
    

In [26]:
get_corr_bootstrap(rets, 2)

0.4546871277867021

In [27]:
num_bootstrap = 100000
corr_seq = np.zeros(num_bootstrap)

t = time.time()
for i in range(num_bootstrap):
    corr_seq[i] = get_corr_bootstrap(rets, i)
print(time.time() - t)

corr_seq = pd.Series(corr_seq)
print(corr_seq.mean())
print(corr_seq.quantile((0.05,0.95)))

12.151875972747803
0.4786298434338419
0.05    0.408004
0.95    0.545673
dtype: float64


In [28]:
#corr_seq.hist(bins=50)

#### Can we use ```Parallel``` to speed up the above Bootstrapping?

In [30]:
t = time.time()
corr_seq2 = pd.Series(Parallel(n_jobs=-1)(delayed(get_corr_bootstrap)(rets, i) for i in range(0, num_bootstrap)) )
print(time.time() - t)

print(corr_seq2.mean())
print(corr_seq2.quantile((0.05,0.95)))

3.231416940689087
0.4786298434338419
0.05    0.408004
0.95    0.545673
dtype: float64


In [31]:
#corr_seq2.hist(bins=50)

---

# END